In [1]:
# test_local_infer.py
import requests, io, json
from pathlib import Path
from PIL import Image
import torch, torch.nn.functional as F
import numpy as np
import timm, albumentations as A
from albumentations.pytorch import ToTensorV2

MODEL_PATH = r"C:\all\infosys_intern_project\models\best_top1_90.4645_ep5.pth"  # set exact path
CLASS_IDX = r"C:\all\infosys_intern_project\json_files\class2idx.json"
IMG_URL = "https://brit-petfood.com/file/nodes/product/bedlingtonterier.jpg"

with open(CLASS_IDX) as f:class2idx = json.load(f)
idx2class = {int(v):k for k,v in class2idx.items()}

# load model
model = timm.create_model('efficientnetv2_rw_s.ra2_in1k', pretrained=False, num_classes=len(class2idx))
ck = torch.load(str(MODEL_PATH), map_location='cpu')
# prefer ema merged earlier; try to load 'model_state' or raw dict
state = ck.get('model_state', ck)
# if merged exists in notebook, you may reuse it instead
model.load_state_dict(state if isinstance(state, dict) else ck, strict=False)
model.eval()

# fetch image
resp = requests.get(IMG_URL, timeout=20)
img = Image.open(io.BytesIO(resp.content)).convert("RGB")
# plt.imshow(img); plt.axis('off'); plt.show()

# preprocess
INPUT_SIZE=384
tf = A.Compose([A.Resize(INPUT_SIZE, INPUT_SIZE), A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)), ToTensorV2()])
img_np = np.array(img)
img_t = tf(image=img_np)['image'].unsqueeze(0)  # 1,C,H,W

# predict
with torch.no_grad():
    out = model(img_t)
    probs = F.softmax(out, dim=1).numpy()[0]
top5_idx = probs.argsort()[-5:][::-1]
print("Top-5 predictions:")
for i in top5_idx:
    print(f"{idx2class[i]}: {probs[i]*100:.2f}%")
print()
print("Top-1:", idx2class[top5_idx[0]])

Top-5 predictions:
bedlington_terrier: 73.97%
saluki: 0.59%
french_bulldog: 0.58%
clumber: 0.52%
border_terrier: 0.51%

Top-1: bedlington_terrier


In [5]:
# ============================
# NEW MODEL LOCAL INFERENCE
# ============================

import io, json, requests
from PIL import Image
import torch
import torch.nn.functional as F
import timm
import torchvision.transforms as T

# ----------------------------
# PATHS (CHANGE IF NEEDED)
# ----------------------------
MODEL_PATH = r"C:\all\infosys_intern_project\models\dog_breed_model_1.pth"
CLASS_IDX = r"C:\all\infosys_intern_project\json_files\class2idx.json"

IMG_URL = "https://petzpark.com.au/cdn/shop/articles/Breeds_Thumbnails_5_1_de5eccaf-bf53-4b93-9173-3284060b4b12_900x.jpg?v=1765442646"

device = "cpu"

# ----------------------------
# LOAD CLASS INDEX
# ----------------------------
with open(CLASS_IDX) as f:
    class2idx = json.load(f)

idx2class = {int(v): k for k, v in class2idx.items()}
num_classes = len(class2idx)

# ----------------------------
# LOAD MODEL
# ----------------------------
model = timm.create_model(
    "mobilenetv3_large_100",
    pretrained=False,
    num_classes=num_classes
)

ckpt = torch.load(MODEL_PATH, map_location=device)
state = ckpt.get("state", ckpt)   # IMPORTANT
model.load_state_dict(state)
model.to(device)
model.eval()

print("✅ Model loaded")

# ----------------------------
# PREPROCESS (MATCH TRAINING)
# ----------------------------
preprocess = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

# ----------------------------
# LOAD IMAGE
# ----------------------------
resp = requests.get(IMG_URL, timeout=20)
img = Image.open(io.BytesIO(resp.content)).convert("RGB")

# ----------------------------
# PREDICT
# ----------------------------
x = preprocess(img).unsqueeze(0).to(device)

with torch.no_grad():
    logits = model(x)
    probs = torch.softmax(logits, dim=1)[0]

top5_vals, top5_idx = torch.topk(probs, 5)

print("\nTop-5 predictions:")
for v, i in zip(top5_vals, top5_idx):
    print(f"{idx2class[i.item()]}: {v.item()*100:.2f}%")

print("\nTop-1:", idx2class[top5_idx[0].item()])

✅ Model loaded

Top-5 predictions:
french_bulldog: 62.10%
pug: 22.88%
bull_mastiff: 5.61%
brabancon_griffon: 3.73%
staffordshire_bullterrier: 1.09%

Top-1: french_bulldog


In [8]:
# test_predict_local.py
import requests, io, json
from PIL import Image
import numpy as np

# path to a sample image (use same example URL or local file)
IMG_URL = "https://brit-petfood.com/file/nodes/product/bedlingtonterier.jpg"
# IMG_URL = "https://i.guim.co.uk/img/media/327aa3f0c3b8e40ab03b4ae80319064e401c6fbc/377_133_3542_2834/master/3542.jpg?width=620&dpr=2&s=none&crop=none"

resp = requests.get(IMG_URL, timeout=20)
img_bytes = resp.content

# call your FastAPI predict endpoint
r = requests.post("http://127.0.0.1:8000/api/predict/", files={"file": ("dog.jpg", img_bytes, "image/jpeg")}, data={"topk": "5"})
print("Status:", r.status_code)
print(r.json())

Status: 200
{'is_dog': True, 'predictions': [{'breed': 'bedlington terrier', 'confidence': 0.7396541833877563}]}


In [3]:
import requests

API_URL = "http://127.0.0.1:8000/api/chat/message"

# Text message test
payload = {
    # "message": "give me the diet plan for rottweiler adult dog",
    # "message": "can we give chocolates to dogs?",
    "message": "what is the climate in australia?"
}

response = requests.post(API_URL, data=payload)

print("Status:", response.status_code)
print("Response:", response.json())


Status: 200
Response: {'predicted_breed': None, 'breed_used': None, 'answer': 'I can only help with dog-related questions.', 'source_data_used': {'breed_provided': False, 'diet_provided': False}}
